In [37]:
import pandas as pd
import numpy as np

In [40]:
NO_atualizado = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=download",
    sheet_name="NO",
    usecols=[
        "NO",
        "NO Ajuste",
        "Categoria",
        "Tipo Valor",
    ]
)
unidade_atualizada = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=download",
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Núcleo 2024",
        "Marca 2024",
        "Unidade 2024",
        "Regional",
    ]
)
coligada_atualizada = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=download",
    sheet_name="Coligadas",
    usecols=[
        "Filial",
        "Marca",
        "Unidade",
    ]
)

def ajuste_NO(df_antigo):
    df = df_antigo.copy()
    df_merge = pd.merge(df, NO_atualizado, on="NO", how="left")
    df_merge = df_merge.drop_duplicates()
    return df_merge

def ajuste_nucleo(df_antigo):
    df = df_antigo.copy()
    df["Chave 2024"] = df["Marca"] + df["Unidade"]
    df_merge = pd.merge(df, unidade_atualizada, on="Chave 2024", how="left")
    df_merge = df_merge.drop_duplicates()
    return df_merge

def ajuste_coligada(df_antigo):
    df = df_antigo.copy()
    df_merge = pd.merge(df, coligada_atualizada, on="Filial", how="left")
    df_merge = df_merge.drop_duplicates()
    return df_merge

In [43]:
REALIZADO_2024 = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1rFuIVjNT6eo5wUr2nL7Y0uvM6r9eXgcaZPrA8Y91OsI/export?=download",
    sheet_name="Consolidado Central",
    usecols=[
        "Contrato",
        "Marca",
        "Unidade",
        "Valor",
        "NO",
        "Mês",
        "Nº FLUIG",
    ]
)

ERRO_REALIZADO_S_MARCA = REALIZADO_2024.loc[(REALIZADO_2024["Marca"].isna()) & (REALIZADO_2024["Valor"] > 0)]

REALIZADO_2024 = REALIZADO_2024[REALIZADO_2024["Marca"].notna()]
REALIZADO_2024.loc[REALIZADO_2024["Marca"].str.startswith("Elite"), "Marca"] = "Elite"
REALIZADO_2024["Valor"] = REALIZADO_2024["Valor"].fillna(0.0)
REALIZADO_2024 = REALIZADO_2024.loc[REALIZADO_2024["Valor"] != 0]

In [34]:
REALIZADO_2024 = ajuste_NO(REALIZADO_2024)
REALIZADO_2024 = ajuste_nucleo(REALIZADO_2024)
REALIZADO_2024

,Nº FLUIG,Contrato,Valor,Marca,Unidade,NO,Mês,NO Ajuste_x,Categoria_x,Tipo Valor_x,...,Marca 2024_x,Unidade 2024_x,Regional_x,NO Ajuste_y,Categoria_y,Tipo Valor_y,Núcleo 2024_y,Marca 2024_y,Unidade 2024_y,Regional_y
0,905279.0,CS0171,41600.00,Coleguium,Carlos Prates,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,...,Coleguium,Carlos Prates,1 - SP/MG/ES/PA,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Coleguium,Carlos Prates,1 - SP/MG/ES/PA
1,905284.0,CS0172,14000.00,Coleguium,Gutierrez,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,...,Coleguium,Gutierrez,1 - SP/MG/ES/PA,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Coleguium,Gutierrez,1 - SP/MG/ES/PA
2,905283.0,CS0173,35200.00,Coleguium,Gutierrez,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,...,Coleguium,Gutierrez,1 - SP/MG/ES/PA,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Coleguium,Gutierrez,1 - SP/MG/ES/PA
3,905525.0,CS0032,70500.00,Coleguium,Alípio de Melo,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,...,Coleguium,Alípio de Melo,1 - SP/MG/ES/PA,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Coleguium,Alípio de Melo,1 - SP/MG/ES/PA
4,905718.0,CS0044,21600.00,Coleguium,Internacional Belvedere,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,...,Coleguium,Internacional Belvedere,1 - SP/MG/ES/PA,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Coleguium,Internacional Belvedere,1 - SP/MG/ES/PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18837,1010138.0,CA1594,1491.23,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,...,Pensi,Campos,5 - RJ,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Pensi,Campos,5 - RJ
18838,1018490.0,CA1595,118.12,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,...,Pensi,Campos,5 - RJ,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Pensi,Campos,5 - RJ
18839,1018492.0,CA1596,69.20,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,...,Pensi,Campos,5 - RJ,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Pensi,Campos,5 - RJ
18840,1012363.0,CA1527,2395.10,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,...,Pensi,Campos,5 - RJ,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Pensi,Campos,5 - RJ


In [33]:
REALIZADO_2024.loc[(REALIZADO_2024["Marca 2024"].isna()) | (REALIZADO_2024["Marca 2024"].isna() != "A definir")]

,Nº FLUIG,Contrato,Valor,Marca,Unidade,NO,Mês,NO Ajuste,Categoria,Tipo Valor,Chave 2024,Núcleo 2024,Marca 2024,Unidade 2024,Regional
0,905279.0,CS0171,41600.00,Coleguium,Carlos Prates,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,ColeguiumCarlos Prates,Núcleo Coleguium,Coleguium,Carlos Prates,1 - SP/MG/ES/PA
1,905284.0,CS0172,14000.00,Coleguium,Gutierrez,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,ColeguiumGutierrez,Núcleo Coleguium,Coleguium,Gutierrez,1 - SP/MG/ES/PA
2,905283.0,CS0173,35200.00,Coleguium,Gutierrez,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,ColeguiumGutierrez,Núcleo Coleguium,Coleguium,Gutierrez,1 - SP/MG/ES/PA
3,905525.0,CS0032,70500.00,Coleguium,Alípio de Melo,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,ColeguiumAlípio de Melo,Núcleo Coleguium,Coleguium,Alípio de Melo,1 - SP/MG/ES/PA
4,905718.0,CS0044,21600.00,Coleguium,Internacional Belvedere,Obra de Reforma,1.0,Obra de Reforma,Infraestrutura,CAPEX,ColeguiumInternacional Belvedere,Núcleo Coleguium,Coleguium,Internacional Belvedere,1 - SP/MG/ES/PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17335,1010138.0,CA1594,1491.23,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,PensiAlpha Campos dos Goytacazes,Núcleo Pensi,Pensi,Campos,5 - RJ
17336,1018490.0,CA1595,118.12,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,PensiAlpha Campos dos Goytacazes,Núcleo Pensi,Pensi,Campos,5 - RJ
17337,1018492.0,CA1596,69.20,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,PensiAlpha Campos dos Goytacazes,Núcleo Pensi,Pensi,Campos,5 - RJ
17338,1012363.0,CA1527,2395.10,Pensi,Alpha Campos dos Goytacazes,Alimentação dos Alunos - Integral,6.0,Alimentação dos Alunos - Integral,Supply,OPEX,PensiAlpha Campos dos Goytacazes,Núcleo Pensi,Pensi,Campos,5 - RJ


In [36]:
REALIZADO_2024.loc[REALIZADO_2024["Marca 2024"].isna()]

KeyError: 'Marca 2024'